In [1]:
#insert json
import json
import requests
import numpy as np
import matplotlib.pyplot as plt
from nltk.tokenize import TreebankWordTokenizer

In [2]:
#load json data
url = "http://api.apomden.com/v2/facilities"
hospitals=requests.get(url).json()
hospitals = hospitals["data"]

### Search Algorithm

In [54]:
SEARCH_FIELDS = ["name", "street", "city", "district", "regions", "services"]

def getTokens(sentence, tokenizer):
    return tokenizer.tokenize(sentence)

def extractJson(list_dicts, field_list):
    
    json_tokens = {field:[] for field in field_list}
    
    for item in list_dicts:
        for field in SEARCH_FIELDS:
            x = extract(item, field)
            json_tokens[field].extend(x)
    return json_tokens

def extract(obj, key):
    
    tokens = []
    
    if isinstance(obj,dict):
        for k,v in obj.items():
            #print(k,v)
            if k==key:
               # print(k)
                tokens.append(v)
            elif isinstance(v, dict):
#                 print(k)
#                 print(v.keys())
#                 print("    ")
                if k in v.keys():
                    print("yes")
                    tokens.extend(extract(v, key))
            elif isinstance(v,(dict,list)):
                tokens.extend(extract(v,key))
                
    elif isinstance(obj, list):
        for item in obj:
            tokens.extend(extract(item,key))
            
    return tokens


def vectorizeTokens(token_dict):
    
    """
    Returns a numpy matrix of vectorized tokens
    """
    
    pass


def buildIndices(token_list):
    
    """
    Returns dictionaries of variable to idx and vice versa pairing
    """
              
    var_to_index = {}
    index_to_var = {}

    for idx, item in enumerate(token_list):
        var_to_index[item] = idx
        index_to_var[idx] = item

    return var_to_index, index_to_var



    

In [55]:
def get_all_tokens(d):
    if not(isinstance(d, list)) and not(isinstance(d, dict)):
        return [d]
    tokens = []
    if isinstance(d, list):
        for i in d:
            tokens.extend(get_all_tokens(i))
    elif isinstance(d, dict):
        for k, v in d.items():
            tokens.extend(get_all_tokens(v))
    return tokens

def extract2(d, key):
    tokens = []
    if isinstance(d, list):
        for i in d:
            tokens.extend(get_all_tokens(i))
    elif isinstance(d, dict):
        for k, v in d.items():
            if k == key:
                tokens.extend(get_all_tokens(v))
            else:
                tokens.extend(extract2(v, key))
    return tokens
            


In [56]:
def print_dict(d):
    for k,v in d.items():
        print(f"------ {k} -----")
        if isinstance(v, list):
            for i in v:
                print(i)
        else:
            print(v)
        print("\n")

In [57]:
def main():
    treebank = TreebankWordTokenizer()
    tokens = extractJson(hospitals, SEARCH_FIELDS)
    print_dict(tokens)

In [58]:
if __name__ == "__main__":
    
    main()

------ name -----
Cocoa Clinic
GW-1-B1
GW-1-B2
GW-1-B3
General Ward - Room 1
GW-2-B1
GW-2-B2
General Ward - Room 2
GW-3-B1
GW-3-B2
GW-3-B3
GW-3-B4
General Ward - Room 3
General Ward
ICU-1-B1
ICU-1-B2
ICU-1-B3
ICU - Room 1
ICU-2-B1
ICU-2-B2
ICU - Room 2
ICU-3-B1
ICU-3-B2
ICU-3-B3
ICU-4-B4
ICU - Room 3
Intensive Care Unit (ICU)
General Primary Care
General Consultation
ICU
Sam Hospital
GW-1-B1
GW-1-B2
GW-1-B3
General Ward - Room 1
GW-2-B1
GW-2-B2
General Ward - Room 2
GW-3-B1
GW-3-B2
GW-3-B3
GW-3-B4
General Ward - Room 3
General Ward
ICU-1-B1
ICU-1-B2
ICU-1-B3
ICU - Room 1
ICU-2-B1
ICU-2-B2
ICU - Room 2
ICU-3-B1
ICU-3-B2
ICU-3-B3
ICU-4-B4
ICU - Room 3
Intensive Care Unit (ICU)
General Primary Care
General Consultation
ICU
Edgar's Hospital
GW-1-B1
GW-1-B2
GW-1-B3
General Ward - Room 1
GW-2-B1
GW-2-B2
General Ward - Room 2
GW-3-B1
GW-3-B2
GW-3-B3
GW-3-B4
General Ward - Room 3
General Ward
ICU-1-B1
ICU-1-B2
ICU-1-B3
ICU - Room 1
ICU-2-B1
ICU-2-B2
ICU - Room 2
ICU-3-B1
ICU-3-B2
ICU-3-B3
ICU-